# TicTacToe

## First Day

For the frist day, we are creating a simple game for two persons. 

## Second Day

In part 2, we will add a few features before creating a bot: 

1) Clean up the code a bit. 
2) Add a status bar and a reset button to start new game. 
3) Include a check_win function to check who the winner and end the game. 

```
[Status]
 ___________
| O |   |   |
 --- --- ---
|   | X |   |
 --- --- ---
| X | O |   |
 --- --- ---
|   Reset   |
 -----------
```

In [1]:
# Install ipywidgets and IPython in they are not found
import ipywidgets as widgets
from IPython.display import display


# The game class to construct the tic tac toe game
class TicTacToe: 
    def __init__(self): 
        self.board = [ [' ' for col in range(3)] for row in range(3)]
        self.player = 'X'
        self.winner = ' '
        
        # User Interface element
        # Status Bar
        self.status = widgets.Label('Ready')
        # Cell Buttons
        self.buttons = [ [widgets.Button(description='') for button in range(3)] for row in range(3)]
        # register the make_move function to each button's onclick event
        for i in range(3): 
            for j in range(3): 
                self.buttons[i][j].on_click(self.make_move(i, j))
        self.button_list = [button for row in self.buttons for button in row]
        # Reset Button
        self.reset_button = widgets.Button(description='New Game', layout=widgets.Layout(width='450px'))
        self.reset_button.on_click(self.reset())
        # Output
        self.output = widgets.Output()
    
    # set a bot to play this game
    def set_bot(self, bot): 
        self.bot = bot
    
    # Start a NEW game
    def reset(self): 
        def on_reset_clicked(_): 
            self.start_over()
        return on_reset_clicked          

    # reset variable to start a new game
    def start_over(self): 
        self.player = 'X'
        self.winner = ' '
        self.status.value = 'Ready'
        # clear the memory of 3x3 matrix
        for i in range(3): 
            for j in range(3): 
                self.board[i][j] = ' '
        # clear the buttons on the grid
        for button in self.button_list: 
            button.description = ' '
    
    # Put either "X" or "O" on the ith row and jth column
    def make_move(self, i, j): 
        def on_button_clicked(_): 
            # human move
            self.move(i,j)
            # bot move
            if self.winner==' ': 
                self.bot.move()
            
        return on_button_clicked
    
    # core function to make a move
    # for a human (click) or a bot
    def move(self, i, j):
        if self.winner==' ' and self.board[i][j] == ' ': 
            self.board[i][j] = self.player
            self.buttons[i][j].description = self.player

            # turn taking
            if self.player == 'X': 
                self.player = 'O'
            else: 
                self.player = 'X'
        self.status.value = 'In progress, ' + self.player + ' playing.'
        # check winner
        self.winner = self.check_win()
        if self.winner != ' ': 
            self.status.value = self.winner + ' won!'
    
    # Check if there is a winner
    # return the winner, 'X' or 'O'
    # OR, return ' ' if no winner
    def check_win(self): 
        # check diagnals
        if self.board[1][1]!=' ' and self.board[0][0]==self.board[1][1] and self.board[1][1]==self.board[2][2]:
            return self.board[0][0]
        if self.board[1][1]!=' ' and self.board[0][2]==self.board[1][1] and self.board[1][1]==self.board[2][0]:
            return self.board[1][1]
        
        # check rows
        for i in range(3): 
            if self.board[i][0]!=' ' and self.board[i][0]==self.board[i][1] and self.board[i][1]==self.board[i][2]:
                return self.board[i][0]
        
        # check columns
        for j in range(3): 
            if self.board[0][j]!=' ' and self.board[0][j]==self.board[1][j] and self.board[1][j]==self.board[2][j]:
                return self.board[0][j]
        
        # no winner found at this point
        return ' '
        
    
    def display(self): 
        self.grid = widgets.GridBox(self.button_list,layout=widgets.Layout( grid_template_columns="repeat(3, 150px)"))
        self.game_box = widgets.VBox([self.status, self.grid, self.reset_button])
        display(self.game_box, self.output)

## Bot 1: Random Bot

The Tic-Tac-Toe game is now ready and can be played by two human players. 

Let's create our first bot so we can play against it--

1. In this first attempt, we will create a **RANDOM** bot just like an iRobot Roomba. 
2. It won't be very smart but will perform some (random) work, again, just like iRobot. 
3. This is our **baseline** model. We will create more advanced (smarter) models down the load and compare them back to this very first model. 

In [84]:
game = TicTacToe()
game.display()

Output()

In [2]:
import random

class RandomBot:
    def __init__(self, game, player): 
        if player not in ['X', 'O']:
            raise ValueError("Player must be either X or O!")
        self.game = game
        self.player = player
    
    # a move function to pick a random cell
    def move(self): 
        avail_cells = [ (i,j) for i in range(3) for j in range(3) if self.game.board[i][j]==' ' ]
        cell = random.choice(avail_cells)
        self.game.move(cell[0], cell[1])

In [86]:
bot = RandomBot(game, 'O')
game.set_bot(bot)

## Bot 2: Learn by Rewards (or Penalty)

### Bug Fix First

In the `check_win()` function: 
1. Check whether there are 3 SAME marks in a (horizontal/vertical/diagonal) row, e.g. 

```python
self.board[0][0]==self.board[1][1] and self.board[1][1]==self.board[2][2]
```

2. In addition, these should be NON-empty marks, i.e.: 

```python
self.board[0][0]!=' ' 
```

### Reinforcement Learning

The following video explains the idea of Reinforcement Learning (RL): 
https://youtu.be/QUPpKgXJd5M?si=D3yRIdaC9GWveXgH

Key ideas of RL applied to Tic-Tac-Toe bot: 
1. Monitors every state of the game, i.e. 'X', 'O', and ' ' marks on the 3x3 grid (`3^9 = 19683` permutations). 
2. Makes a move and, depending on the `exploration rate`, it will select: 
    * EITHER an random move to **explore** different situations
    * OR the **best move** based on past rewards
3. No reward or penalty if there is no immediate winner. 
4. In the end: 
    * IF the bot wins, the LAST move will receive a reward of `1`
    * IF the bot loses, the LAST move will receive a penalty of `-1`
5. This repeat with NEW games and the bot continues to learn. 

Reading the above, one may wonder whether ONLY the LAST move will be rewarded? The answer is NO. 
* (1) **All actions leading to** the LAST move (for a win or loss) will be rewarded or penalized but the reward/penalty will be **discounted**. 
```
        For example, given RLBot Actions: 
        Move1 => Move2 => Move3 (WIN)

        Its rewards will be like: 

        Move3 (1 point) =>  Move2 (0.9 point) => Move1 (0.9*0.9 point)
```

* (2) We do have to **repeat the game** to train the bot in order to update rewards/penalities to previous moves. 
* (3) Another parameter `learning rate` determines **how fast** the bot will update the reward/penalty. 

In [87]:
game = TicTacToe()
game.display()
bot = RandomBot(game, 'O')
game.set_bot(bot)

Output()

In [88]:
np_board2 = np.array(game.board)
np_board1 = np_board2.reshape(-1)
str(np_board1)

"[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']"

In [27]:
list(zip(*np.where(np_board2 == ' ')))

[(0, 1), (0, 2), (1, 0), (2, 0), (2, 2)]

In [89]:
avail_actions = list(zip(*np.where(np_board2 == ' ')))
avail_indices = [np.ravel_multi_index(action, (3, 3)) for action in avail_actions]
avail_indices

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [3]:
import numpy as np

class RLBot:
    def __init__(self, game, player, learning_rate=0.5, discount_factor=0.9, exploration_rate=0.1):
        self.game = game
        self.player = player
        self.q_table = {}
        self.set_params(learning_rate, discount_factor, exploration_rate)
    
    # change game
    def change_game(self, game): 
        self.game = game
    
    # parameters affecting the bot's learning behavior
    def set_params(self, learning_rate=0.5, discount_factor=0.9, exploration_rate=0.1): 
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        
    # board is a 2-dimensional np.array of 'X', 'O', or ' ' values
    def get_state(self, board):
        # board = np.array(self.game.board)
        return str(board.reshape(-1))  # flatten to 1-dimensional
        
    # get current q values of a given `state`
    def get_q_values(self, state):
        if state not in self.q_table:
            self.q_table[state] = np.zeros(9)
        return self.q_table[state]
    
    # get available actions
    def get_avail_actions(self, board): 
        actions_2d = list(zip(*np.where(board == ' ')))
        actions_1d = [np.ravel_multi_index(act, (3, 3)) for act in actions_2d]
        return actions_1d
    
    # a potential draw when board is full
    def board_is_full(self, board): 
        return len(self.get_avail_actions(board))==0
    
    # select an action given a `state` and `available actions`
    def select_action(self, state, available_actions):
        if np.random.random() < self.exploration_rate:
            action = np.random.choice(available_actions)
        else: 
            q_values = self.get_q_values(state)
            # Get the q_values of the available actions
            available_q_values = q_values[available_actions]
            # select the action with the highest available q value
            best_action_index = np.argmax(available_q_values)
            # return to available_actions to find the original index for the best value
            action = available_actions[best_action_index]
        return action
    
    # update q values for the (old) state and action
    # based on 1) immediate reward, and 2) existing reward in the new state (that the action leads to)
    def update_q_values(self, old_state, action, reward, new_state):
        old_q_values = self.get_q_values(old_state)
        new_q_values = self.get_q_values(new_state)
        old_q_values[action] = old_q_values[action] + self.learning_rate * (reward + self.discount_factor * np.max(new_q_values) - old_q_values[action])
        
    # make a move, an exploratory (random) or learned one. 
    # return [old_state, action, new_state]
    def move(self):
        board = np.array(self.game.board)
        state = self.get_state(board)
        actions_1d = self.get_avail_actions(board)
        action_1d = self.select_action(state, actions_1d)
        action_2d = np.unravel_index(action_1d, (3,3))
        # print(action_1d, action_2d)
        self.game.move(action_2d[0], action_2d[1])
        new_board = np.array(self.game.board)
        new_state = self.get_state(new_board)
        return state, action_1d, new_state

In [174]:
game = TicTacToe()
game.display()
bot = RLBot(game, 'O', 0.5, 0.9, 1)
game.set_bot(bot)

Output()

In [175]:
game = TicTacToe()
Simon = RLBot(game, 'X', 0.5, 0.8, 1)
Olive = RLBot(game, 'O', 0.5, 0.8, 1)
game.display()

Output()

In [116]:
Rex.move()

6 (2, 0)


("['X' 'O' 'X' 'O' 'X' 'O' ' ' ' ' ' ']",
 6,
 "['X' 'O' 'X' 'O' 'X' 'O' 'X' ' ' ' ']")

In [117]:
Lea.move()

7 (2, 1)


("['X' 'O' 'X' 'O' 'X' 'O' 'X' ' ' ' ']",
 7,
 "['X' 'O' 'X' 'O' 'X' 'O' 'X' ' ' ' ']")

In [4]:
# game and bots
game = TicTacToe()
Simon = RLBot(game, 'X', 0.5, 0.8, 1)
Olive = RLBot(game, 'O', 0.5, 0.8, 1)

# tracking arrays
players = ['X', 'O']
bots = [Simon, Olive]
old_states = ['','']
actions = [0, 0]
new_states = ['','']

# train the bots
def train(): 
    # in game variable
    in_game = True
    game.start_over()
    play_idx = 0
    while in_game: 

        old_state, action, new_state = bots[play_idx].move()
        old_states[play_idx] = old_state
        new_states[play_idx] = new_state
        actions[play_idx] = action

        # check win or end
        board = np.array(game.board)
        winner = game.check_win()
        if winner!=' ': 
            in_game = False
            for i in range(2): 
                # reward the winner
                if players[i]==winner: 
                    bots[i].update_q_values(old_states[i], actions[i], 10, new_states[i])
                # penalize the loser
                else: 
                    bots[i].update_q_values(old_states[i], actions[i], -10, new_states[i])
        elif playing_bot.board_is_full(board): 
            in_game = False

        # alternate turn
        play_idx = 1 if play_idx==0 else 0

In [191]:
# game.display()

In [179]:
train()

In [154]:
o_state, act, n_state = Simon.move()

In [158]:
o_state, act, n_state = Olive.move()

In [164]:
Simon.update_q_values(o_state, act, 10, n_state) 

In [165]:
Olive.update_q_values(o_state, act, 10, n_state) 

In [171]:
Olive.get_q_values(o_state)

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [167]:
Simon.get_q_values(o_state)

array([0. , 0. , 0. , 0. , 0. , 1.9, 0. , 0. , 0. ])

In [149]:
old_states, actions, new_states

(["['O' 'X' 'X' 'X' 'O' 'O' ' ' ' ' ' ']",
  "['O' 'X' 'X' 'X' 'O' 'O' 'X' ' ' ' ']"],
 [6, 8],
 ["['O' 'X' 'X' 'X' 'O' 'O' 'X' ' ' ' ']",
  "['O' 'X' 'X' 'X' 'O' 'O' 'X' ' ' 'O']"])

In [189]:
Human_vs_Olive = TicTacToe()
Human_vs_Olive.display()
Human_vs_Olive.set_bot(Olive)
Olive.set_params(0.5, 0.9, 0.1)
Olive.change_game(Human_vs_Olive)

Output()

In [134]:
Lea.move()

7 (2, 1)


("['X' 'O' 'X' 'O' 'X' 'O' 'X' ' ' ' ']",
 7,
 "['X' 'O' 'X' 'O' 'X' 'O' 'X' ' ' ' ']")

## Invisible Game

For training

In [5]:
# The game class to construct the tic tac toe game
class MemoryTicTacToe: 
    def __init__(self): 
        self.board = [ [' ' for col in range(3)] for row in range(3)]
        self.player = 'X'
        self.winner = ' '
    
    # set a bot to play this game
    def set_bot(self, bot): 
        self.bot = bot

    # reset variable to start a new game
    def start_over(self): 
        self.player = 'X'
        self.winner = ' '
        # clear the memory of 3x3 matrix
        for i in range(3): 
            for j in range(3): 
                self.board[i][j] = ' '
    
    # core function to make a move
    # for a human (click) or a bot
    def move(self, i, j):
        if self.winner==' ' and self.board[i][j] == ' ': 
            self.board[i][j] = self.player

            # turn taking
            if self.player == 'X': 
                self.player = 'O'
            else: 
                self.player = 'X'
        # check winner
        self.winner = self.check_win()

    
    # Check if there is a winner
    # return the winner, 'X' or 'O'
    # OR, return ' ' if no winner
    def check_win(self): 
        # check diagnals
        if self.board[1][1]!=' ' and self.board[0][0]==self.board[1][1] and self.board[1][1]==self.board[2][2]:
            return self.board[0][0]
        if self.board[1][1]!=' ' and self.board[0][2]==self.board[1][1] and self.board[1][1]==self.board[2][0]:
            return self.board[1][1]
        
        # check rows
        for i in range(3): 
            if self.board[i][0]!=' ' and self.board[i][0]==self.board[i][1] and self.board[i][1]==self.board[i][2]:
                return self.board[i][0]
        
        # check columns
        for j in range(3): 
            if self.board[0][j]!=' ' and self.board[0][j]==self.board[1][j] and self.board[1][j]==self.board[2][j]:
                return self.board[0][j]
        
        # no winner found at this point
        return ' '
        

In [9]:
# game and bots
game = MemoryTicTacToe()
Simon = RLBot(game, 'X', 0.1, 0.9, 1)
Olive = RLBot(game, 'O', 0.1, 0.9, 1)

# tracking arrays
players = ['X', 'O']
bots = [Simon, Olive]
old_states = ['','']
actions = [0, 0]
new_states = ['','']

# train the bots
def train(): 
    # in game variable
    in_game = True
    game.start_over()
    play_idx = 0
    while in_game: 

        old_state, action, new_state = bots[play_idx].move()
        old_states[play_idx] = old_state
        new_states[play_idx] = new_state
        actions[play_idx] = action

        # check win or end
        board = np.array(game.board)
        winner = game.check_win()
        if winner!=' ': 
            in_game = False
            for i in range(2): 
                # reward the winner
                if players[i]==winner: 
                    bots[i].update_q_values(old_states[i], actions[i], 10, new_states[i])
                # penalize the loser
                else: 
                    bots[i].update_q_values(old_states[i], actions[i], -10, new_states[i])
        elif bots[play_idx].board_is_full(board): 
            in_game = False

        # alternate turn
        play_idx = 1 if play_idx==0 else 0

In [10]:
import time

start_time = time.time()
for run in range(10000000): 
    train()
end_time = time.time()
elapsed = (end_time - start_time)*1000
print(f"The program took {elapsed} milliseconds to complete.")

The program took 4743447.942733765 milliseconds to complete.


In [11]:
real_game = TicTacToe()
real_game.display()
real_game.set_bot(Olive)
Olive.set_params(0.5, 0.9, 0)
Olive.change_game(real_game)

Output()